In [136]:
import pandas as pd
import numpy as np
from datasets import  load_from_disk
from pathlib import Path


In [137]:


def read_and_procc_geometry(fn = Path("../data/sensor_geometry.csv")):
    sensor_geometry_df = pd.read_csv("../data/sensor_geometry.csv")
        # counts
    doms_per_string = 60
    string_num = 86

    # index
    outer_long_strings = np.concatenate(
        [np.arange(0, 25), np.arange(27, 34), np.arange(37, 44), np.arange(46, 78)]
    )
    inner_long_strings = np.array([25, 26, 34, 35, 36, 44, 45])
    inner_short_strings = np.array([78, 79, 80, 81, 82, 83, 84, 85])

    # known specs
    outer_xy_resolution = 125.0 / 2
    inner_xy_resolution = 70.0 / 2
    long_z_resolution = 17.0 / 2
    short_z_resolution = 7.0 / 2

    # evaluate error
    sensor_x = sensor_geometry_df.x
    sensor_y = sensor_geometry_df.y
    sensor_z = sensor_geometry_df.z
    sensor_r_err = np.ones(doms_per_string * string_num)
    sensor_z_err = np.ones(doms_per_string * string_num)

    for string_id in outer_long_strings:
        sensor_r_err[
            string_id * doms_per_string : (string_id + 1) * doms_per_string
        ] *= outer_xy_resolution
    for string_id in np.concatenate([inner_long_strings, inner_short_strings]):
        sensor_r_err[
            string_id * doms_per_string : (string_id + 1) * doms_per_string
        ] *= inner_xy_resolution

    for string_id in outer_long_strings:
        sensor_z_err[
            string_id * doms_per_string : (string_id + 1) * doms_per_string
        ] *= long_z_resolution
    for string_id in np.concatenate([inner_long_strings, inner_short_strings]):
        for dom_id in range(doms_per_string):
            z = sensor_z[string_id * doms_per_string + dom_id]
            if (z < -156.0) or (z > 95.5 and z < 191.5):
                sensor_z_err[string_id * doms_per_string + dom_id] *= short_z_resolution
    # register
    sensor_geometry_df["r_err"] = sensor_r_err
    sensor_geometry_df["z_err"] = sensor_z_err

    # detector constants
    c_const = 0.299792458  # speed of light [m/ns]

    x_min = sensor_x.min()
    x_max = sensor_x.max()
    y_min = sensor_y.min()
    y_max = sensor_y.max()
    z_min = sensor_z.min()
    z_max = sensor_z.max()

    detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
    t_valid_length = detector_length / c_const

    print("t_valid_length: ", t_valid_length, " ns")

    return sensor_geometry_df



In [138]:
sensor_geometry_df = read_and_procc_geometry()

t_valid_length:  6199.700247193777  ns


In [139]:
# read single event from batch_meta_df
def perprocess_event(
    event_feature,
    max_pulse_count=128,
    t_valid_length=6199.700247193777,
    sensor_geometry_df=sensor_geometry_df,
):

    event_feature = pd.DataFrame(
        {
            "event_id": event_feature["event_id"],
            "sensor_id": event_feature["sensor_id"],
            "time": event_feature["time"],
            "charge": event_feature["charge"],
            "auxiliary": event_feature["auxiliary"],
            "x": event_feature["x"],
            "y": event_feature["y"],
            "z": event_feature["z"],
        }
    )

    sensor_id = event_feature["sensor_id"]

    # merge features into single structured array
    dtype = [
        ("time", "float16"),
        ("charge", "float16"),
        ("auxiliary", "float16"),
        ("x", "float16"),
        ("y", "float16"),
        ("z", "float16"),
        ("r_err", "float16"),
        ("z_err", "float16"),
        ("rank", "short"),
    ]
    event_x = np.zeros(len(event_feature), dtype)

    event_x["time"] = event_feature.time.values - event_feature.time.min()
    event_x["charge"] = event_feature.charge.values
    event_x["auxiliary"] = event_feature.auxiliary.values

    event_x["x"] = sensor_geometry_df.x[sensor_id].values
    event_x["y"] = sensor_geometry_df.y[sensor_id].values
    event_x["z"] = sensor_geometry_df.z[sensor_id].values

    event_x["r_err"] = sensor_geometry_df.r_err[sensor_id].values
    event_x["z_err"] = sensor_geometry_df.z_err[sensor_id].values

    # For long event, pick-up
    if len(event_x) > max_pulse_count:
        # Find valid time window
        t_peak = event_x["time"][event_x["charge"].argmax()]
        t_valid_min = t_peak - t_valid_length
        t_valid_max = t_peak + t_valid_length

        t_valid = (event_x["time"] > t_valid_min) * (event_x["time"] < t_valid_max)

        # rank
        event_x["rank"] = 2 * (1 - event_x["auxiliary"]) + (t_valid)

        # sort by rank and charge (important goes to backward)
        event_x = np.sort(event_x, order=["rank", "charge"])

        # pick-up from backward
        event_x = event_x[-max_pulse_count:]

        # resort by time
        event_x = np.sort(event_x, order="time")

    # for train data, give angles together
    event_x = {name: event_x[name] for name in (event_x.dtype.names)}
    return {"event": event_x}


def preprocess_v2(batch, max_pulse_count=128, t_valid_length=6199.700247193777):
    col = batch.columns
    t_peak = batch["time"][batch["charge"].argmax()]
    t_valid_min = t_peak - t_valid_length
    t_valid_max = t_peak + t_valid_length
    t_valid = (batch["time"] > t_valid_min) * (batch["time"] < t_valid_max)
    batch["rank"] = 2 * (1 - batch["auxiliary"]) + (t_valid)
    batch = batch.sort_values(by=["rank", "charge"])
    # pick-up from backward
    batch = batch[-max_pulse_count:]
        # resort by time
    batch = batch.sort_values(by="time")
    return batch[col]
        
        


In [140]:
batch = load_from_disk('/opt/slh/icecube/data/hf_cashe/batch_1.parquet')

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [215]:
pd.DataFrame(batch[4])[['x', 'y', 'z', 'charge', 'time']][:5]

,x,y,z,charge,time
0,79.41,-248.24,464.86,0.825,6083
1,41.60,35.49,-473.29,1.275,6194
2,-66.70,276.92,140.39,0.525,6567
3,114.39,-461.99,-267.27,1.025,7003
4,46.29,-34.88,-264.96,0.625,7200


In [141]:
bin_num = 16
azimuth_edges = np.linspace(0, 2 * np.pi, bin_num + 1)
zenith_edges_flat = np.linspace(0, np.pi, bin_num + 1)
zenith_edges = list()
zenith_edges.append(0)
for bin_idx in range(1, bin_num):
    # cos(zen_before) - cos(zen_now) = 2 / bin_num
    zen_now = np.arccos(np.cos(zenith_edges[-1]) - 2 / (bin_num))
    zenith_edges.append(zen_now)
zenith_edges.append(np.pi)
zenith_edges = np.array(zenith_edges)

In [142]:
def y_to_onehot(batch_y):
    # evaluate bin code
    azimuth_code = (batch_y[:, 0] > azimuth_edges[1:].reshape((-1, 1))).sum(axis=0)
    zenith_code = (batch_y[:, 1] > zenith_edges[1:].reshape((-1, 1))).sum(axis=0)
    angle_code = bin_num * azimuth_code + zenith_code

    # one-hot
    batch_y_onehot = np.zeros((angle_code.size, bin_num * bin_num))
    batch_y_onehot[np.arange(angle_code.size), angle_code] = 1
    
    return batch_y_onehot

angle_bin_zenith0 = np.tile(zenith_edges[:-1], bin_num)
angle_bin_zenith1 = np.tile(zenith_edges[1:], bin_num)
angle_bin_azimuth0 = np.repeat(azimuth_edges[:-1], bin_num)
angle_bin_azimuth1 = np.repeat(azimuth_edges[1:], bin_num)

angle_bin_area = (angle_bin_azimuth1 - angle_bin_azimuth0) * (np.cos(angle_bin_zenith0) - np.cos(angle_bin_zenith1))
angle_bin_vector_sum_x = (np.sin(angle_bin_azimuth1) - np.sin(angle_bin_azimuth0)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_y = (np.cos(angle_bin_azimuth0) - np.cos(angle_bin_azimuth1)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_z = (angle_bin_azimuth1 - angle_bin_azimuth0) * ((np.cos(2 * angle_bin_zenith0) - np.cos(2 * angle_bin_zenith1)) / 4)

angle_bin_vector_mean_x = angle_bin_vector_sum_x / angle_bin_area
angle_bin_vector_mean_y = angle_bin_vector_sum_y / angle_bin_area
angle_bin_vector_mean_z = angle_bin_vector_sum_z / angle_bin_area

angle_bin_vector = np.zeros((1, bin_num * bin_num, 3))
angle_bin_vector[:, :, 0] = angle_bin_vector_mean_x
angle_bin_vector[:, :, 1] = angle_bin_vector_mean_y
angle_bin_vector[:, :, 2] = angle_bin_vector_mean_z

def pred_to_angle(pred, epsilon=1e-8):
    # convert prediction to vector
    pred_vector = (pred.reshape((-1, bin_num * bin_num, 1)) * angle_bin_vector).sum(axis=1)
    
    # normalize
    pred_vector_norm = np.sqrt((pred_vector**2).sum(axis=1))
    mask = pred_vector_norm < epsilon
    pred_vector_norm[mask] = 1
    
    # assign <1, 0, 0> to very small vectors (badly predicted)
    pred_vector /= pred_vector_norm.reshape((-1, 1))
    pred_vector[mask] = np.array([1., 0., 0.])
    
    # convert to angle
    azimuth = np.arctan2(pred_vector[:, 1], pred_vector[:, 0])
    azimuth[azimuth < 0] += 2 * np.pi
    zenith = np.arccos(pred_vector[:, 2])
    
    return azimuth, zenith

In [143]:
index = 656
label = np.array([batch[23]['azimuth'], batch[23]['zenith']]).reshape(1, -1)
print(label)
label[:, 0] += azimuth_edges[1] - azimuth_edges[0]
label[:, 0][label[:, 0] > 2 * np.pi] -= 2 * np.pi
y_to_onehot(label)

[[4.16105643 1.42740734]]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [135]:
az, zen = pred_to_angle(y_to_onehot(label))
az -= azimuth_edges[1] - azimuth_edges[0]
az, zen

(array([4.12334036]), array([1.38086169]))

In [183]:
import torch

data = torch.tensor([[303.41, 335.64, 206.58, 1.325, 5928],
                     [-145.45, 374.24, 212.73, 1.175, 6115],
                     [505.27, 257.88, -174.60, 0.925, 6492],
                     [-9.68, -79.50, 181.00, 0.225, 6665],
                     [576.37, 170.92, 357.88, 1.575, 8054]])

In [220]:
import torch

def calculate_features(data):
    # Calculate the distance of each interaction point from the IceCube detector
    distance = np.sqrt(np.sum(data[:, :3]**2, axis=1))
    
    # Calculate the time differences between the arrival times of the different signals
    time_differences = np.diff(data[:, 4])
    
    # Calculate the charge ratios of the different signals
    charge_ratios = np.diff(data[:, 3]) / data[:-1, 3]
    
    # Combine the additional features into a single array
    
    
    return distance, time_differences, charge_ratios

In [221]:
data = torch.tensor([[303.41, 335.64, 206.58, 1.325, 5928],
                     [-145.45, 374.24, 212.73, 1.175, 6115],
                     [505.27, 257.88, -174.60, 0.925, 6492],
                     [-9.68, -79.50, 181.00, 0.225, 6665],
                     [576.37, 170.92, 357.88, 1.575, 8054]])





In [225]:
calculate_features(data.numpy())[1].shape

(4,)

In [219]:
data.numpy()

array([[ 3.0341e+02,  3.3564e+02,  2.0658e+02,  1.3250e+00,  5.9280e+03],
       [-1.4545e+02,  3.7424e+02,  2.1273e+02,  1.1750e+00,  6.1150e+03],
       [ 5.0527e+02,  2.5788e+02, -1.7460e+02,  9.2500e-01,  6.4920e+03],
       [-9.6800e+00, -7.9500e+01,  1.8100e+02,  2.2500e-01,  6.6650e+03],
       [ 5.7637e+02,  1.7092e+02,  3.5788e+02,  1.5750e+00,  8.0540e+03]],
      dtype=float32)